In [16]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.optimizers import Adam
from keras.models import model_from_json

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Load data

### Webcam dataset

In [2]:
X_raw = np.load("data/images.npy")
y = np.load("data/labels.npy")
y = [val[0] for val in y]

X = []
for x in X_raw:
    X.append(preprocess_input(x))

### Shanghai tech B and density maps

### Train:

In [18]:
Train_imgs_B = []
Train_density_B = []
Train_labels_B = []

path_images = "./data/ShanghaiTech/part_B/train_data/images/"
path_heads = "./data/ShanghaiTech/part_B/train_data/ground-truth/"
path_maps = "./data/ShanghaiTech/part_B/train_data/maps/"

for img_name, den_name, heads_name in zip(os.listdir(path = path_images), os.listdir(path = path_maps), os.listdir(path = path_heads)):   
    
    Train_imgs_B.append(preprocess_input(cv2.cvtColor(cv2.imread(path_images + img_name), cv2.COLOR_BGR2RGB)))
    Train_density_B.append(np.load(path_maps + den_name))
    Train_labels_B.append(len(loadmat(path_heads + heads_name)['image_info'][0, 0][0, 0][0]))

### Test:

In [17]:
Test_imgs_B = []
Test_density_B = []
Test_labels_B = []

path_images = "./data/ShanghaiTech/part_B/test_data/images/"
path_heads = "./data/ShanghaiTech/part_B/test_data/ground-truth/"
path_maps = "./data/ShanghaiTech/part_B/test_data/maps/"

for img_name, den_name, heads_name in zip(os.listdir(path = path_images), os.listdir(path = path_maps), os.listdir(path = path_heads)):   
    
    Test_imgs_B.append(preprocess_input(cv2.cvtColor(cv2.imread(path_images + img_name), cv2.COLOR_BGR2RGB)))
    Test_density_B.append(np.load(path_maps + den_name))
    Test_labels_B.append(len(loadmat(path_heads + heads_name)['image_info'][0, 0][0, 0][0]))

## Build the model

In [4]:
VGG_16 = VGG16(weights='imagenet', include_top=False, input_shape=(None, None, 3))

In [15]:
Map_estimator_net = Sequential([
            Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, padding = 'same'),
            Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, padding = 'same'),
            Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, padding = 'same'),
            Conv2D(256, (3, 3), activation='relu', dilation_rate = 2, padding = 'same'),
            Conv2D(128, (3, 3), activation='relu', dilation_rate = 2, padding = 'same'),
            Conv2D(64, (3, 3), activation='relu', dilation_rate = 2, padding = 'same'),
            Conv2D(1, (1, 1), activation='relu', dilation_rate = 1, padding = 'same')
            ])(VGG_16.output)

CSRnet = Model(inputs=VGG_16.input, outputs=Map_estimator_net)
CSRnet.compile(optimizer=Adam(), loss = "mse", metrics=["mse","mae"])
CSRnet.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   

I use the last 50 images of the train set as validation set

In [26]:
history = CSRnet.fit(Train_imgs_B[:-50], Train_density_B[:-50],
                             batch_size = 1, epochs=1,
                             validation_data=(Train_imgs_B[-50:],Train_density_B[-50:]))

MemoryError: Unable to allocate 9.00 MiB for an array with shape (768, 1024, 3) and data type float32

316